# Extract Canadian Postal Code from Wikipedia 

Install BeatifulSoup

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         516 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0

The following packages will be UPDATED:

  ca-certificate

Import libraries and packages

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

Load page  [List of postal codes of Canada: M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) and create BeautifulSoup object

In [3]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', 'html.parser')
soup = BeautifulSoup(response.text, 'html.parser')

Function to clean html tags and newline character

In [4]:
def clean_data(data, tag):
    return str(data).replace('<' + tag + '>','').replace('</' + tag + '>','').replace("\n",'').strip()

Navigate through the BeautifulSoup object and extract column names and data

In [5]:
rows = []
column_names = []
header = True
for tr in soup.find_all('tr'):
    if header:
        for th in tr:
            if not('\n' in th):
                column_names.append(clean_data(th,'th'))
        header = False
    else:
        cols = []
        for td in tr:
            if not('\n' in td):
                cols.append(clean_data(td,'td'))
        if len(cols)==3: # and (not "Not assigned" in cols[1]):
            rows.append(cols) #{column_names[0] : cols[0], column_names[1] : cols[1], column_names[2] : cols[2]})

construct data frame based in table data extracted from html page

In [6]:
df = pd.DataFrame(rows, columns  = column_names)
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.

In [7]:
df.drop(df[df.Borough == "Not assigned"].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough.

In [8]:
df[df.Neighborhood == "Not assigned"]

,Postal Code,Borough,Neighborhood


In [9]:
df.Neighborhood = df.Borough.where(df.Neighborhood == 'Not assigned', df.Neighborhood)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [10]:
df_grouped = df.groupby('Postal Code').Neighborhood.apply(lambda x : ', '.join(x.tolist())).to_frame() # group same postal code neighborhoods
df_temp = df.drop_duplicates(subset=['Postal Code']) # dropo rows with same postal code
df_area = pd.merge(df_temp[['Postal Code', 'Borough']], df_grouped[['Neighborhood']], on='Postal Code', how='right') # merge Postal Code and Borough with respective neighborhood
df_area

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
df_area.shape

(103, 3)